In [2]:
from tensorflow import keras
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
import json
import os
pd.set_option('display.max_colwidth', None)

In [7]:
# function to load training & test sets into DataFrames
def load_data(filename : str):
    meta = pd.read_csv(filename).reset_index()
    actors = meta['actor_name']
    paths = meta['path']
    meta = np.vstack((actors, paths))
    meta_df = pd.DataFrame(meta).T
    meta_df.columns = ['actor_name', 'path']
    return meta_df

# predict a class using img file
def predict_class(filepath):
    import cv2
    img = cv2.imread(filepath)
    resized_img = cv2.resize(img, (128, 128))
    normalizedImg = np.zeros((800, 800))
    normalizedImg = cv2.normalize(resized_img, normalizedImg, 0, 255, cv2.NORM_MINMAX)
    resized_img = normalizedImg.reshape(1, 128, 128, 3)
    return lookup_table[str(np.argmax(model.predict(resized_img)))]

def evaluate_model(test_set, lookup_table):
    # set the truth list and convert to ints
    truth = test_set['actor_name']
    name_to_int = dict(map(reversed, lookup_table.items()))
    truth = [name_to_int[name] for name in truth]
    
    # create paths of images to predict
    images = test_set['path']
    images = [path.replace('.\data\lfw-deepfunneled', '../../../data/lfw-deepfunneled') for path in images]
    
    predictions = []
    for image in images:
        predictions.append(name_to_int[predict_class(image)])
        
    return classification_report(truth, predictions, labels=np.unique(predictions))

In [5]:
## load data of model
model = keras.models.load_model('8_epoch70.h5')
train_set = load_data('data/train_set.csv')
test_set = load_data('data/test_set.csv')
lookup_table = {}
with open('data/lookup_table.json', 'r') as f:
    lookup_table = json.loads(f.read())

In [ ]:
report = evaluate_model(test_set, lookup_table)

1/1 [==============================] - 0s 185ms/step
